# squeezenet1_0
Here, we are going to use squeezenet1_0 model pre-trained on ImageNet, would be used for beans image classification.

## Importing required libraries and functions

In [1]:
import os
import time
import random
from PIL import Image
import numpy as np
from glob import glob

import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score

## Seeding

In [2]:
random.seed(42)
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

## Data Transforms
Define transformations for data augmentation and normalization

In [3]:
# Define transforms for data augmentation and normalization
train_transform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    # transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    # transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

## Split the dataset
Split the dataset into training, validation, and testing sets

In [4]:
dataset_path = "/media/nikhil/New Volume/ML_DATASET/Beans/"

train_ratio = 0.70
valid_ratio = 0.15
test_ratio = 0.15

In [5]:
class_folders = sorted(os.listdir(dataset_path))

In [6]:
class_folders

['Bariah_1',
 'Bariah_2',
 'Bariah_3',
 'Bariah_raw',
 'Harari_1',
 'Harari_2',
 'Harari_3',
 'Harari_raw',
 'Khowlani_1',
 'Khowlani_2',
 'Khowlani_3',
 'Khowlani_raw',
 'Legamti_1',
 'Legamti_2',
 'Legamti_3',
 'Legamti_raw',
 'Nabari_1',
 'Nabari_2',
 'Nabari_3',
 'Nabari_raw']

In [7]:
train_x, valid_x, test_x = [], [], []
train_y, valid_y, test_y = [], [], []

In [8]:
# Iterate through each class folder
for i, class_folder in enumerate(class_folders):
    class_path = os.path.join(dataset_path, class_folder)
    # images = os.listdir(class_path)
    images = sorted(glob(os.path.join(class_path, "*")))
    num_images = len(images)
    
    # Calculate number of samples for each split
    num_train = int(train_ratio * num_images)
    num_valid = int(valid_ratio * num_images)
    num_test = num_images - num_train - num_valid
    
    # Allocate samples to each split
    random.shuffle(images)
    train_samples = images[:num_train]
    valid_samples = images[num_train:num_train + num_valid]
    test_samples = images[num_train + num_valid:]
    
    train_x += train_samples
    valid_x += valid_samples
    test_x += test_samples
    
    train_y += [i] * len(train_samples)
    valid_y += [i] * len(valid_samples)
    test_y += [i] * len(test_samples)

In [9]:
print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_y)}")

Train: 8607/8607 - Valid: 1838/1838 - Test: 1864/1864


## Loading the dataset

In [10]:
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        image = Image.open(image).convert('RGB')
        label = np.array(label)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [11]:
# Create custom datasets with transforms
train_dataset = CustomDataset(train_x, train_y, transform=train_transform)
valid_dataset = CustomDataset(valid_x, valid_y, transform=valid_transform)
test_dataset = CustomDataset(test_x, test_y, transform=test_transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Data Loaders

In [12]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Model

In [13]:
model = torchvision.models.squeezenet1_0(pretrained=True)
# Freeze the pre-trained layers
for param in model.parameters():
    param.requires_grad = True

/home/nikhil/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nikhil/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, dim_feedforward=128, dropout=0.1):
        super().__init__()

        self.attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        ## Reshape
        src = x.view((x.shape[0], x.shape[1], x.shape[2]*x.shape[3]))

        # Self-attention layer
        src2 = self.attn(src, src, src, attn_mask=mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)

        # Feedforward layer
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)

        src = src.view((x.shape[0], x.shape[1], x.shape[2], x.shape[3]))

        return src

In [15]:
model.classifier._modules["1"] = TransformerBlock(169, 13)

model.classifier.append(nn.Conv2d(512, 20, kernel_size=(1, 1)))
model.num_classes = 20
   
for param in model.classifier.parameters():
    param.requires_grad = True

In [16]:
from torchsummary import summary
summary(model.cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 109, 109]          14,208
              ReLU-2         [-1, 96, 109, 109]               0
         MaxPool2d-3           [-1, 96, 54, 54]               0
            Conv2d-4           [-1, 16, 54, 54]           1,552
              ReLU-5           [-1, 16, 54, 54]               0
            Conv2d-6           [-1, 64, 54, 54]           1,088
              ReLU-7           [-1, 64, 54, 54]               0
            Conv2d-8           [-1, 64, 54, 54]           9,280
              ReLU-9           [-1, 64, 54, 54]               0
             Fire-10          [-1, 128, 54, 54]               0
           Conv2d-11           [-1, 16, 54, 54]           2,064
             ReLU-12           [-1, 16, 54, 54]               0
           Conv2d-13           [-1, 64, 54, 54]           1,088
             ReLU-14           [-1, 64,

## Utility Functions

In [17]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Define Loss and Optimizer

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

## Training

In [19]:
# Training loop
num_epochs = 500
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

best_valid_loss = float('inf')
train_losses = []
valid_losses = []
patience = 10

early_stop_counter = 0  # Counter for early stopping

for epoch in range(num_epochs):
    start_time = time.time()
    
    ## Training
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss)
    
    ## Validation
    model.eval()
    valid_loss = 0.0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    
    valid_loss /= len(valid_loader)
    valid_losses.append(valid_loss)
    
    scheduler.step(valid_loss)
    
    ## Print
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    data_str = f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n"
    data_str += f"\tTrain Loss: {train_loss:.4f}\n"
    data_str += f"\tValid Loss: {valid_loss:.4f}\n"
    print(data_str)
    
    ## Check for early stopping
    if valid_loss < best_valid_loss:
        early_stop_counter = 0  # Reset the counter if validation loss improves
        torch.save(model.state_dict(), 'squeezenet1_0_best_weights.pth')
        
        print(f"Valid loss decreased from {best_valid_loss:1.5f} to {valid_loss:1.5f}. Saving checkpoint.\n")
        
        best_valid_loss = valid_loss
        early_stop_counter = 0
        
    else:
        early_stop_counter += 1
        # print(f"Epoch {epoch+1:1.3f}, Train Loss: {train_loss:1.5f}, Val Loss: {valid_loss:1.5f}")
    
    if early_stop_counter >= patience:
        print(f"Early stopping after {patience} epochs of no improvement.")
        break

Epoch: 01 | Epoch Time: 0m 19s
	Train Loss: 2.9793
	Valid Loss: 2.9835

Valid loss decreased from inf to 2.98350. Saving checkpoint.

Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 2.9579
	Valid Loss: 2.9693

Valid loss decreased from 2.98350 to 2.96934. Saving checkpoint.

Epoch: 03 | Epoch Time: 0m 20s
	Train Loss: 2.9084
	Valid Loss: 2.9306

Valid loss decreased from 2.96934 to 2.93061. Saving checkpoint.

Epoch: 04 | Epoch Time: 0m 19s
	Train Loss: 2.8062
	Valid Loss: 2.8068

Valid loss decreased from 2.93061 to 2.80675. Saving checkpoint.

Epoch: 05 | Epoch Time: 0m 20s
	Train Loss: 2.6123
	Valid Loss: 2.4821

Valid loss decreased from 2.80675 to 2.48208. Saving checkpoint.

Epoch: 06 | Epoch Time: 0m 22s
	Train Loss: 2.2850
	Valid Loss: 2.1719

Valid loss decreased from 2.48208 to 2.17192. Saving checkpoint.

Epoch: 07 | Epoch Time: 0m 21s
	Train Loss: 2.0174
	Valid Loss: 2.0087

Valid loss decreased from 2.17192 to 2.00867. Saving checkpoint.

Epoch: 08 | Epoch Time: 0m 21s
	Train 

Epoch: 73 | Epoch Time: 0m 19s
	Train Loss: 0.1266
	Valid Loss: 0.5309

Valid loss decreased from 0.54100 to 0.53087. Saving checkpoint.

Epoch: 74 | Epoch Time: 0m 19s
	Train Loss: 0.1256
	Valid Loss: 0.5377

Epoch: 75 | Epoch Time: 0m 19s
	Train Loss: 0.1259
	Valid Loss: 0.5236

Valid loss decreased from 0.53087 to 0.52359. Saving checkpoint.

Epoch: 76 | Epoch Time: 0m 19s
	Train Loss: 0.1252
	Valid Loss: 0.5313

Epoch: 77 | Epoch Time: 0m 19s
	Train Loss: 0.1242
	Valid Loss: 0.5284

Epoch: 78 | Epoch Time: 0m 19s
	Train Loss: 0.1251
	Valid Loss: 0.5262

Epoch: 79 | Epoch Time: 0m 19s
	Train Loss: 0.1239
	Valid Loss: 0.5139

Valid loss decreased from 0.52359 to 0.51390. Saving checkpoint.

Epoch: 80 | Epoch Time: 0m 19s
	Train Loss: 0.1239
	Valid Loss: 0.5147

Epoch: 81 | Epoch Time: 0m 19s
	Train Loss: 0.1233
	Valid Loss: 0.5067

Valid loss decreased from 0.51390 to 0.50672. Saving checkpoint.

Epoch: 82 | Epoch Time: 0m 19s
	Train Loss: 0.1237
	Valid Loss: 0.5082

Epoch: 83 | Epoc

Epoch: 150 | Epoch Time: 0m 19s
	Train Loss: 0.1107
	Valid Loss: 0.4101

Epoch: 151 | Epoch Time: 0m 19s
	Train Loss: 0.1105
	Valid Loss: 0.4074

Epoch: 152 | Epoch Time: 0m 19s
	Train Loss: 0.1102
	Valid Loss: 0.3998

Valid loss decreased from 0.40491 to 0.39979. Saving checkpoint.

Epoch: 153 | Epoch Time: 0m 19s
	Train Loss: 0.1100
	Valid Loss: 0.4048

Epoch: 154 | Epoch Time: 0m 19s
	Train Loss: 0.1098
	Valid Loss: 0.3991

Valid loss decreased from 0.39979 to 0.39913. Saving checkpoint.

Epoch: 155 | Epoch Time: 0m 19s
	Train Loss: 0.1098
	Valid Loss: 0.3993

Epoch: 156 | Epoch Time: 0m 19s
	Train Loss: 0.1095
	Valid Loss: 0.4053

Epoch: 157 | Epoch Time: 0m 19s
	Train Loss: 0.1093
	Valid Loss: 0.3967

Valid loss decreased from 0.39913 to 0.39674. Saving checkpoint.

Epoch: 158 | Epoch Time: 0m 19s
	Train Loss: 0.1095
	Valid Loss: 0.3939

Valid loss decreased from 0.39674 to 0.39387. Saving checkpoint.

Epoch: 159 | Epoch Time: 0m 19s
	Train Loss: 0.1088
	Valid Loss: 0.3982

Epoch:

## Saving weight file and training data

In [20]:
losses_df = pd.DataFrame({'Train Loss': train_losses, 'Val Loss': valid_losses})
losses_df.to_csv('training_val_losses.csv', index=False)

## Testing

In [21]:
model = torchvision.models.squeezenet1_0(pretrained=False)
model.classifier._modules["1"] = TransformerBlock(169, 13)
model.classifier.append(nn.Conv2d(512, 20, kernel_size=(1, 1)))
model.num_classes = 20
    
model.load_state_dict(torch.load('squeezenet1_0_best_weights.pth'))  # Load the trained weights
model.to(device)
model.eval()  # Set the model to evaluation mode

/home/nikhil/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [22]:
true_labels = []
predicted_labels = []

In [23]:
# Iterate through the test dataset and make predictions
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

In [24]:
# Calculate accuracy, recall, precision, F1 score, and F2 score
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels, average='macro')
precision = precision_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')
f2 = fbeta_score(true_labels, predicted_labels, beta=2, average='macro')

print(f"Test Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")
print(f"F2 Score: {f2}")

Test Accuracy: 0.8342274678111588
Recall: 0.8479217534894665
Precision: 0.84521074198137
F1 Score: 0.8458910638716507
F2 Score: 0.8469488625878577
